In [1]:
import spacy
import torch
import dspy
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import ndcg_score
import numpy as np
import random
import faiss
from sklearn.preprocessing import normalize
random.seed(42)
import spacy
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
def preprocess_text(text):
    text = text.lower()
    
    text = re.sub(r'[^\w\s]', '', text)
    
    tokens = word_tokenize(text)
    
    stop_words = set(stopwords.words('english'))
    
    tokens = [token for token in tokens if token not in stop_words]
    
    ps = PorterStemmer()
    
    tokens = [ps.stem(token) for token in tokens]
    
    return tokens

In [3]:
nlp = spacy.load('en_core_web_sm')

def replace_names_with_placeholder(text):
    doc = nlp(text)
    processed_text = []

    for token in doc:
        if token.ent_type_ == "PERSON": 
            processed_text.append("name")  
        elif token.ent_type_ == "NORP":
            processed_text.append("group") 
        elif token.ent_type_ == "ORG":
            processed_text.append("organisation") 
        else:
            processed_text.append(token.text)

    return ' '.join(processed_text)


In [4]:
tell_me_again_df = pd.read_csv("data/tell_me_again.csv")
tell_me_again_df['label'] = tell_me_again_df.groupby('title').ngroup()
tell_me_again_df['property_count'] = tell_me_again_df['label'].map(tell_me_again_df['label'].value_counts())

In [5]:
# only select summaries if there are 2 or more summaries of the same movie
tell_me_again_df = tell_me_again_df[tell_me_again_df.property_count >= 2]

In [6]:
summary_indices = random.sample(list(set(tell_me_again_df.label)),1000)

In [7]:
tell_me_again_df = tell_me_again_df[tell_me_again_df.label.isin(summary_indices)]
#tell_me_again_df['processed_summary'] = tell_me_again_df['unpacked_summary'].apply(replace_names_with_placeholder)
tell_me_again_df

,Unnamed: 0,wikidata_id,title,language,unpacked_summary,unpacked_summary_sents,label,property_count
48,48,Q100265988,Black Box,de,"In a car accident, young mother Rachel is kill...","['In a car accident, young mother Rachel is ki...",2800,5
49,49,Q100265988,Black Box,it,Nolan is a man who suffers from severe amnesia...,['Nolan is a man who suffers from severe amnes...,2800,5
61,61,Q100349419,Home,de,Marvin Hacks is riding his skateboard on the h...,['Marvin Hacks is riding his skateboard on the...,9026,9
62,62,Q100349419,Home,fr,Marvin Hacks (Jake McLaughlin) returns to his ...,['Marvin Hacks (Jake McLaughlin) returns to hi...,9026,9
77,77,Q1004440,Kill the Irishman,de,Danny Greene grows up as an Irish orphan in Cl...,['Danny Greene grows up as an Irish orphan in ...,10665,4
...,...,...,...,...,...,...,...,...
83126,83126,Q992813,"Ride, Vaquero!",de,"After the Civil War, American farmers settled ...","['After the Civil War, American farmers settle...",16105,3
83127,83127,Q992813,"Ride, Vaquero!",fr,"In Texas, just after the Civil War, a bandit n...","['In Texas, just after the Civil War, a bandit...",16105,3
83128,83128,Q992813,"Ride, Vaquero!",it,Mexican bandit José Esqueda and his gang conti...,['Mexican bandit José Esqueda and his gang con...,16105,3
83228,83229,Q998048,Trimurti,de,Satyadevi is an honest policewoman and mother ...,['Satyadevi is an honest policewoman and mothe...,25450,2


In [8]:
tell_me_again_df.unpacked_summary.iloc[1]

'Nolan is a man who suffers from severe amnesia due to a terrible car accident in which his wife lost her life. Despite severe memory problems, he continues to live with his baby daughter Ava, aided only by his best friend Gary, an orthopedic doctor. The man finds himself completely managed by his child and loses his job as a photojournalist because his shots are no longer up to his old works. Nolan also develops some characteristics that he never had in the past: he forgets to pick up his daughter from school and has small bouts of violence in which he loses control. He also begins to resent some of the rituals of his family routine that are dear to his daughter. In an attempt to regain his memory, Nolan, after several unsuccessful attempts with traditional medicine, begins experimental therapy with Dr. Brooks, who has developed a special hypnosis strategy that allows him to relive his most important memories.\nNolan\'s visions, however, are different from what he expected: the man se

In [9]:
labels = tell_me_again_df.label.tolist()
texts = tell_me_again_df.unpacked_summary.tolist()

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device=device)
embedder = dspy.Embedder(model.encode)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\emiel\.conda\envs\DSP-project\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\emiel\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
embeddings = embedder(texts)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi

def build_index(embeddings, normalize_vectors=True):
    if normalize_vectors:
        return normalize(embeddings, axis=1)
    return embeddings

def get_dense_similarity(query_vector, candidate_vectors):
    return np.dot(candidate_vectors, query_vector.T).flatten()

def get_bm25_scores(corpus, query, bm25=None):
    if bm25 is None:
        bm25 = BM25Okapi([doc for doc in corpus])
    return bm25.get_scores(query)

def get_relevance_scores(i, labels, data, index, k=None, method='dense', bm25=None):
    if k is None:
        k = len(labels)

    query_label = labels[i]
    matching_labels = np.where(np.array(labels) == query_label)[0]

    if method == 'dense':
        query_vector = index[i].reshape(1, -1)
        similarities = get_dense_similarity(query_vector, index)
    elif method == 'bm25':
        query_text = data[i]
        similarities = get_bm25_scores(data, query_text, bm25)
    else:
        raise ValueError("Unsupported method. Use 'dense' or 'bm25'.")

    top_k_indices = np.argsort(similarities)[::-1]
    top_k_indices = top_k_indices[top_k_indices != i][:k]

    relevance_scores = [1 if x in matching_labels else 0 for x in top_k_indices]
    ideal_relevance_scores = sorted(relevance_scores, reverse=True)

    return ideal_relevance_scores, relevance_scores

def calculate_mean_ndcg_score(labels, data, embeddings=None, method='dense'):
    if method == 'dense':
        embeddings = np.array(embeddings).astype('float32')
        index = build_index(embeddings)
        get_index = lambda: index
        bm25 = None
    elif method == 'bm25':
        index = data
        get_index = lambda: index
        bm25 = BM25Okapi([doc for doc in data])
    else:
        raise ValueError("Method must be 'dense' or 'bm25'.")

    ideal_relevance_scores = []
    relevance_scores = []

    for i in [labels.index(x) for x in list(set(labels))]:
        irs, rs = get_relevance_scores(i, labels, data, get_index(), method=method, bm25=bm25)
        ideal_relevance_scores.append(irs)
        relevance_scores.append(rs)

    return ndcg_score(np.array(ideal_relevance_scores), np.array(relevance_scores))


In [14]:
%%time
ndcg_bm25 = calculate_mean_ndcg_score(labels, data=[preprocess_text(text) for text in texts], method='bm25')
ndcg_bm25

CPU times: total: 2min 7s
Wall time: 2min 6s


0.8567717232009066

In [15]:
ndcg_dense = calculate_mean_ndcg_score(labels, data=texts, embeddings=embeddings, method='dense')
ndcg_dense

0.8453171331711882

# 2. Zero-shot Ollama + DSPy

In [16]:
text = tell_me_again_df.unpacked_summary.iloc[65]

In [17]:
instructions = "Your task is to extract elements from a narrative, if there is no instance of a specific element in the text return None"

In [18]:
#text = text + "\n" + instructions

In [19]:
lm = dspy.LM('ollama_chat/gemma3:1b', api_base='http://localhost:11434', api_key='')
dspy.configure(lm=lm)

In [20]:
class ExtractNarrativeElements(dspy.Signature):
    text: str = dspy.InputField()
    
    time_periods: list[str] = dspy.OutputField(desc = """Establishes the historical context and influences character behavior, societal norms, technology, and major historical events. Key elements include specific historical era or time frame, including contemporary, historical, and futuristic settings.""")
    locations: list[str] = dspy.OutputField(desc = """Shapes the environment and specific challenges faced by characters, often serving as a critical component of the plot. Key element include physical locations such as continents, countries, cities, and specific environments like jungles, deserts, and urban areas.""")




module = dspy.Predict(ExtractNarrativeElements)

response = module(text=text)

In [21]:
response.locations

['Paris', "Brignon's apartment", 'various Parisian streets and alleyways']

In [22]:
text

"Paris, December of '46. Jenny Lamour (Delair) wants to make it in the theater. Her husband and companion is Maurice Martineau (Blier), a quiet but jealous man. When she learns that Jenny has been eyeing Brignon, an old and lustful businessman, in order to further her career, she loses her temper and threatens Brignon with death. Despite this, Jenny goes on a secret rendezvous at Brignon's apartment, but is nevertheless murdered that same night. Criminal investigations are in charge of Inspector Antoine (Jouvet)."